# 네이버 뉴스 크롤링

Last update: May 28. 2017  
Kyusik Kim

* 깔끔하지도 않고 오직 나만을 위해 만든 크롤러이기 때문에 몇 가지 TIP을 공유하는 차원에서 글을 쓴다. (공유라기 보다는 그냥 내가 까먹지 않으려고... 일을 그만 두면서 코드를 가지고 왔다고 생각했는데, 감쪽같이 사라져 새로 만드는 데 난감했기 때문이다)
* 먼저 크롤링(crawling)은 우리가 인터넷 서핑할 때 보는 웹 페이지 내에 있는 데이터를 추출해 내는 행위를 말하며, 이를 수행하는 일련의 시스템(?)을 크롤러(crawler)라 할 수 있겠다. 다양한 방법이 있지만, 나는 Python을 이용한다. 
* Python을 이용해 크롤링을 할 때 핵심이 되는 라이브러리를 열거하면, BeautifulSoup, requests, urllib.request 등이 있다. 보조적으로, pandas, re, time, random 을 이용한다.  
![그림1](https://github.com/iamglenn/mypython/blob/master/pic/pic_crawl_1.png?raw=true)


* 네이버 뉴스의 경우 기사 제목인 "포천시~ 참가" 부분과 빨간색 네모박스의 "국제뉴스" 부분의 url이 다르다. 둘의 차이를 설명하면, 기사 제목을 누르면 국제뉴스와 연결되고, 국제뉴스를 누르면 네이버뉴스 플랫폼에서 제공하는 국제뉴스의 기사에 연결된다는 점이 다르다. 이런 것은 좀 더 편하게 크롤링을 하기 위한 꼼수라고 해야 할까?
* 순서는 아래의 내용을 참고

* **import module**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
import urllib.request
import re
import time
import random

* **네이버 뉴스 url 입력하기**

In [2]:
url_basic = "http://news.naver.com/main/search/search.nhn?query=%BE%C6%B4%C2%C7%FC%B4%D4&st=news.all&q_enc=EUC-KR&r_enc=UTF-8&r_format=xml&rp=none&sm=all.basic&ic=all&so=rel.dsc&rcnews=exist:032:005:086:020:021:081:022:023:025:028:038:469:&rcsection=exist:&stDate=range:20170521:20170528&detail=0&pd=3&r_cluster2_start=11&r_cluster2_display=10&start=11&display=10&startDate=2017-05-21&endDate=2017-05-28&page=1"
url_basic = url_basic[:-1]

* **총 페이지 수**

In [3]:
total_pages = 1

* **내보낼 때 파일 이름**

In [4]:
file_name = r'\아는형님'

In [5]:
article_url = []
press_name = []
press_time = []
press_title = []
press_text = []

* **url 가져오기**

In [6]:
url = url_basic + str(1)
source_code_from_URL = urllib.request.urlopen(url)
print(source_code_from_URL)

* **BeautifulSoup으로 가져오기**

In [7]:
soup = BeautifulSoup(source_code_from_URL, "lxml", from_encoding = "utf-8")

* **div의 info class에 해당하는 내용 가져오기**

In [8]:
item = soup.find_all('div', 'info')

* **div class="info"에서 a태그에 달려 있는 각 네이버뉴스의 url 주소 가져오기**

In [9]:
article = item[0].find('a')['href']
print(article)

http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=106&oid=081&aid=0002824625


In [10]:
news_url = urllib.request.urlopen(article)

* **네이버뉴스에서 추출한 url에서 내용을 BeautifulSoup으로 가져오기**

In [11]:
soup2 = BeautifulSoup(news_url, "lxml", from_encoding = "utf-8")

* **네이버뉴스에서 div 태그에 있는 articeBody를 가져오기**
* **articeBody는 본문 내용이며, 이 html태그는 chrome에서 F12를 눌러 확인가능**
* **class 이름은 바뀔 수 있음을 유의. 예를 들면 세계일보는 articeBody, 조선일보는 articleBodyContents를 사용하기도 함**

In [17]:
article_body = soup2.find('div', id='articeBody')

* **텍스트 부분만 가져오기**

In [18]:
dirt_text = article_body.find_all(text=True)

* **html 태그 없애는 정규표현식이었던듯..?**

In [19]:
cleaned_text = re.sub('[a-zA-Z]', '', str(dirt_text))

* **\ 이런 것들 없애주는 정규표현식**

In [20]:
cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                              '', cleaned_text)

* **앞에 달려 있는 "본문 폰트 설정 안내나눔고딕"등의 글들을 없애주는 정규표현식**

In [21]:
cleaned_text = re.sub('본문 폰트 설정  안내나눔고딕  1맑은고딕  2돋움  3바탕  4폰트 사이즈 1  1폰트 사이즈 2  2폰트 사이즈 3  3폰트 사이즈 4  4폰트 사이즈 5  5',
                      '', cleaned_text)

* **기사 제목을 가져오기 위해 p 태그에 있는 end_tit class를 가져오기**
* **텍스트 다듬는 것은 위와 동일**

In [22]:
title = soup2.find('p', attrs={'class':'end_tit'})
print(title)

<p class="end_tit">‘아는형님’ 오현경, “강호동이 이상형..고백했으면 사귀었을 것”</p>


In [23]:
title2 = title.find_all(text=True)
print(title2)                

['‘아는형님’ 오현경, “강호동이 이상형..고백했으면 사귀었을 것”']


In [24]:
title2 = re.sub('[a-zA-Z]', '', str(title2))
print(title2)               

['‘아는형님’ 오현경, “강호동이 이상형..고백했으면 사귀었을 것”']


In [25]:
title2 = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', title2)
print(title2)

‘아는형님’ 오현경 “강호동이 이상형고백했으면 사귀었을 것”


* **기사 정보들 가져오기**
* **신문사, 게시 시간 등**

In [26]:
a = item[0]

In [27]:
press = a.find('span', {'class': 'press'}).text
print(press)

서울신문


In [28]:
date = a.find('span',{'class':'time'}).text
print(date)

3시간전


In [29]:
article_url.append(article)
press_name.append(press)
press_time.append(date)  
press_title.append(title2)
press_text.append(cleaned_text)

* **Series로 만든 후 DataFrame으로 합친 결과를 내보내면 끝**

In [30]:
date = Series(press_time)
name = Series(press_name)
press_url = Series(article_url)
title_sr = Series(press_title)
text_sr = Series(press_text)

url_df = pd.concat([title_sr, name, date, text_sr, press_url], axis=1)
url_df.columns = ['TITLE', 'NAME', 'TIME', 'TEXT', 'URL']

In [31]:
pd.DataFrame(url_df)

,TITLE,NAME,TIME,TEXT,URL
0,‘아는형님’ 오현경 “강호동이 이상형고백했으면 사귀었을 것”,서울신문,3시간전,서울신문 ‘아는형님’ 오현경 배우 오현경이 “강호동이 이상형이다”고 ...,http://news.naver.com/main/read.nhn?mode=LSD&m...


## 완성!

In [40]:
for i in range(1, total_pages+1):
    url = url_basic + str(i)
    
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, "lxml", from_encoding = "utf-8")
    item = soup.find_all('div', 'info')
    
    for a in item:
        try:
            article = a.find('a')['href']
        
            news_url = urllib.request.urlopen(article)
            soup2 = BeautifulSoup(news_url, "lxml", from_encoding = "utf-8")
            article_body = soup2.find('div', id='articleBodyContents')
            dirt_text = article_body.find_all(text=True)
            cleaned_text = re.sub('[a-zA-Z]', '', str(dirt_text))
            cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                              '', cleaned_text)
            cleaned_text = re.sub('본문 내용    플레이어     플레이어     오류를 우회하기 위한 함수 추가',
                         '', cleaned_text)

            title = soup2.find('h3', id='articleTitle')
            title2 = title.find_all(text=True)
            title2 = re.sub('[a-zA-Z]', '', str(title2))
            title2 = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                              '', title2)
            
        except AttributeError:
            try:
                article = a.find('a')['href']
                news_url = urllib.request.urlopen(article)
                soup2 = BeautifulSoup(news_url, "lxml", from_encoding = "utf-8")
                article_body = soup2.find('div', id='articeBody')
                dirt_text = article_body.find_all(text=True)
                cleaned_text = re.sub('[a-zA-Z]', '', str(dirt_text))
                cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                                  '', cleaned_text)
                cleaned_text = re.sub('본문 내용    플레이어     플레이어     오류를 우회하기 위한 함수 추가',
                             '', cleaned_text)

                title = soup2.find('p', attrs={'class':'end_tit'})
                title2 = title.find_all(text=True)
                title2 = re.sub('[a-zA-Z]', '', str(title2))
                title2 = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                                  '', title2)
            
            except AttributeError:
                title2 = "Error"
                cleaned_text = "Error"
            
            
        
        
        
        press = a.find('span', {'class': 'press'}).text
        date = a.find('span',{'class':'time'}).text
    
        article_url.append(article)
        press_name.append(press)
        press_time.append(date)
        
        press_title.append(title2)
        press_text.append(cleaned_text)
        
    random_time = random.randint(20, 40) # 이 부분은 네이버뉴스를 크롤링할 때 페이지 넘기는 속도 제어하는 부분
    time.sleep(random_time)

In [41]:
i

2

In [42]:
date = Series(press_time)
name = Series(press_name)
press_url = Series(article_url)
title_sr = Series(press_title)
text_sr = Series(press_text)

url_df = pd.concat([title_sr, name, date, text_sr, press_url], axis=1)
url_df.columns = ['TITLE', 'NAME', 'TIME', 'TEXT', 'URL']

In [43]:
url_df.to_csv(r"D:\test" + file_name + ".csv")

In [44]:
print("==================Finished=================")

==================Finished=================
